# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
'''
Anpassen: 

number_of_variables --> anzahl variablen
lambda_dataset_size --> datensatz größe für training von lambda-nets
number_of_generated_datasets --> anzahl der lambda-nets

ANMERKUNG: Hier werden die daten von make_classification nach dem generieren in [0,1] skaliert (das wäre evtl sinnvoll bei dir auch zu machen)
'''

'\nAnpassen: \n\nnumber_of_variables --> anzahl variablen\nlambda_dataset_size --> datensatz größe für training von lambda-nets\nnumber_of_generated_datasets --> anzahl der lambda-nets\n\nANMERKUNG: Hier werden die daten von make_classification nach dem generieren in [0,1] skaliert (das wäre evtl sinnvoll bei dir auch zu machen)\n'

In [2]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 10,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,          
        'dt_type': 'vanilla', #'vanilla', 'SDT'
    },
    'data': {
        'number_of_variables': 10, 
        'num_classes': 2,
        'categorical_indices': [],
        'random_parameters_distribution': True, ##MAKEPATH DIFFERENT FILES
        'max_distributions_per_class': 1, # None; 0; int >= 1
        'exclude_linearly_seperable': False,
        'data_generation_filtering': False,
        'fixed_class_probability': False,
        'balanced_data': True,
        'weighted_data_generation': False,
        'shift_distrib': False,
        
        'function_generation_type': 'make_classification' ,#'distribution', 'distribution_trained' 'make_classification_distribution', 'make_classification_distribution_trained', 'make_classification', 'make_classification_trained', 'random_decision_tree', 'random_decision_tree_trained'
        'distribution_list': ['uniform', 'normal', 'gamma', 'beta', 'poisson'],#['uniform', 'normal', 'gamma', 'beta', 'poisson', 'lognormal', 'exponential', 'f', 'logistic', 'weibull'],#['uniform', 'normal', 'gamma', 'exponential', 'beta', 'binomial', 'poisson'], 
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 4_000,
        
        'noise_injected_level': 0, 
        'noise_injected_type': 'flip_percentage', # '' 'normal' 'uniform' 'normal_range' 'uniform_range'
        
        'data_noise': 0, #None or float
        'distrib_param_max': 5,
    }, 
    'computation':{
        'n_jobs': 100,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


In [3]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [4]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter



from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree_BASIC import SDT
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from IPython.display import Image

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [5]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
os.environ['CUDA_VISIBLE_DEVICES'] = ''

sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')



In [6]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

#initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [7]:
print(path_identifier_function_data)

lNetSize5000_numDatasets4000_var10_class2_make_classification_xMax1_xMin0_xDistuniform_dNoise0_randParamDist_maxDistClass1_distribParamMax5_randClassProb


# Function Generation

In [8]:
if function_generation_type == 'random_decision_tree':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))
elif function_generation_type == 'random_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))  
elif function_generation_type == 'make_classification':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    
elif function_generation_type == 'make_classification_distribution':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification_distribution)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_distribution_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification_distribution_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    
elif function_generation_type == 'distribution':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky') #sequential
    result_list = parallel(delayed(generate_data_distribution)(config, 
                                                              seed=RANDOM_SEED+i, 
                                                              max_distributions_per_class=max_distributions_per_class, 
                                                              random_parameters=random_parameters_distribution,
                                                              data_noise=data_noise) for i in range(number_of_generated_datasets))    #, distribution_list = ['uniform']
elif function_generation_type == 'distribution_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_distribution_trained)(config, 
                                                                      seed=RANDOM_SEED+i, 
                                                                      max_distributions_per_class=max_distributions_per_class, 
                                                                      random_parameters=random_parameters_distribution,
                                                                      data_noise=data_noise) for i in range(number_of_generated_datasets))    #, distribution_list = ['uniform']
    
    
function_identifier_list = generate_decision_tree_identifier(config)  
identifier_series_list = [pd.Series(result[0],  index=function_identifier_list) for result in result_list]

function_df = pd.DataFrame(data=np.array([result[0] for result in result_list]), columns=function_identifier_list)

X_data_list = [[identifier_series, pd.DataFrame(result[1], columns=['x' + str(i) for i in range(number_of_variables)])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_list = [[identifier_series, pd.DataFrame(result[2], columns=['result'])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_raw_list = [[identifier_series, pd.DataFrame(result[3], columns=['result_raw'])] for identifier_series, result in zip(identifier_series_list, result_list)]
try:
    distribution_parameter_list_list =[[identifier_series, result[4]] for identifier_series, result in zip(identifier_series_list, result_list)]
except:
    pass

[Parallel(n_jobs=100)]: Using backend LokyBackend with 100 concurrent workers.
[Parallel(n_jobs=100)]: Done  88 tasks      | elapsed:   45.8s
[Parallel(n_jobs=100)]: Done 312 tasks      | elapsed:   46.8s
[Parallel(n_jobs=100)]: Done 600 tasks      | elapsed:   48.0s
[Parallel(n_jobs=100)]: Done 952 tasks      | elapsed:   49.7s
[Parallel(n_jobs=100)]: Done 1368 tasks      | elapsed:   52.6s
[Parallel(n_jobs=100)]: Done 1848 tasks      | elapsed:   58.2s
[Parallel(n_jobs=100)]: Done 2392 tasks      | elapsed:  1.0min
[Parallel(n_jobs=100)]: Done 3000 tasks      | elapsed:  1.1min
[Parallel(n_jobs=100)]: Done 3672 tasks      | elapsed:  1.2min
[Parallel(n_jobs=100)]: Done 4000 out of 4000 | elapsed:  1.4min finished


In [9]:
function_df.head(20)

,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,feat31,feat32,feat33,feat34,feat35,feat36,feat37,feat38,feat39,feat40,feat41,feat42,feat43,feat44,feat45,feat46,feat47,feat48,feat49,feat50,feat51,feat52,feat53,feat54,feat55,feat56,feat57,feat58,feat59,feat60,feat61,feat62,feat63,feat64,feat65,feat66,feat67,feat68,feat69,feat70,feat71,feat72,feat73,feat74,feat75,feat76,feat77,feat78,feat79,feat80,feat81,feat82,feat83,feat84,feat85,feat86,feat87,feat88,feat89,feat90,feat91,feat92,feat93,feat94,feat95,feat96,feat97,feat98,feat99,feat100,feat101,feat102,feat103,feat104,feat105,feat106,feat107,feat108,feat109,feat110,feat111,feat112,feat113,feat114,feat115,feat116,feat117,feat118,feat119,feat120,feat121,feat122,feat123,feat124,feat125,feat126,feat127,feat128,feat129,feat130,feat131,feat132,feat133,feat134,feat135,feat136,feat137,feat138,feat139,feat140,feat141,feat142,feat143,feat144,feat145,feat146,feat147,feat148,feat149,feat150,feat151,feat152,feat153,feat154,feat155,feat156,feat157,feat158,feat159,feat160,feat161,feat162,feat163,feat164,feat165,feat166,feat167,feat168,feat169,feat170,feat171,feat172,feat173,feat174,feat175,feat176,feat177,feat178,feat179,feat180,feat181,feat182,feat183,feat184,feat185,feat186,feat187,feat188,feat189,feat190,feat191,feat192,feat193,feat194,feat195,feat196,feat197,feat198,feat199,feat200,feat201,feat202,feat203,feat204,feat205,feat206,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,feat217,feat218,feat219,feat220,feat221,feat222,feat223,feat224,feat225,feat226,feat227,feat228,feat229,feat230,feat231,feat232,feat233,feat234,feat235,feat236,feat237,feat238,feat239,feat240,feat241,feat242,feat243,feat244,feat245,feat246,feat247,feat248,feat249,...,lp774,lp775,lp776,lp777,lp778,lp779,lp780,lp781,lp782,lp783,lp784,lp785,lp786,lp787,lp788,lp789,lp790,lp791,lp792,lp793,lp794,lp795,lp796,lp797,lp798,lp799,lp800,lp801,lp802,lp803,lp804,lp805,lp806,lp807,lp808,lp809,lp810,lp811,lp812,lp813,lp814,lp815,lp816,lp817,lp818,lp819,lp820,lp821,lp822,lp823,lp824,lp825,lp826,lp827,lp828,lp829,lp830,lp831,lp832,lp833,lp834,lp835,lp836,lp837,lp838,lp839,lp840,lp841,lp842,lp843,lp844,lp845,lp846,lp847,lp848,lp849,lp850,lp851,lp852,lp853,lp854,lp855,lp856,lp857,lp858,lp859,lp860,lp861,lp862,lp863,lp864,lp865,lp866,lp867,lp868,lp869,lp870,lp871,lp872,lp873,lp874,lp875,lp876,lp877,lp878,lp879,lp880,lp881,lp882,lp883,lp884,lp885,lp886,lp887,lp888,lp889,lp890,lp891,lp892,lp893,lp894,lp895,lp896,lp897,lp898,lp899,lp900,lp901,lp902,lp903,lp904,lp905,lp906,lp907,lp908,lp909,lp910,lp911,lp912,lp913,lp914,lp915,lp916,lp917,lp918,lp919,lp920,lp921,lp922,lp923,lp924,lp925,lp926,lp927,lp928,lp929,lp930,lp931,lp932,lp933,lp934,lp935,lp936,lp937,lp938,lp939,lp940,lp941,lp942,lp943,lp944,lp945,lp946,lp947,lp948,lp949,lp950,lp951,lp952,lp953,lp954,lp955,lp956,lp957,lp958,lp959,lp960,lp961,lp962,lp963,lp964,lp965,lp966,lp967,lp968,lp969,lp970,lp971,lp972,lp973,lp974,lp975,lp976,lp977,lp978,lp979,lp980,lp981,lp982,lp983,lp984,lp985,lp986,lp987,lp988,lp989,lp990,lp991,lp992,lp993,lp994,lp995,lp996,lp997,lp998,lp999,lp1000,lp1001,lp1002,lp1003,lp1004,lp1005,lp1006,lp1007,lp1008,lp1009,lp1010,lp1011,lp1012,lp1013,lp1014,lp1015,lp1016,lp1017,lp1018,lp1019,lp1020,lp1021,lp1022,lp1023
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [10]:
X_data_list[0][0].head(10)

feat0    0
feat1    0
feat2    0
feat3    0
feat4    0
feat5    0
feat6    0
feat7    0
feat8    0
feat9    0
dtype: int64

In [11]:
X_data_list[0][1].head(10)

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,0.482,0.502,0.495,0.471,0.670,0.421,0.540,0.557,0.509,0.635
1,0.524,0.748,0.693,0.144,0.580,0.423,0.274,0.609,0.550,0.464
2,0.504,0.687,0.599,0.556,0.632,0.338,0.573,0.449,0.466,0.694
3,0.399,0.522,0.557,0.287,0.415,0.581,0.281,0.519,0.511,0.473
4,0.551,0.657,0.858,0.345,0.575,0.433,0.570,0.494,0.520,0.360
5,0.394,0.472,0.477,0.662,0.685,0.377,0.412,0.672,0.486,0.497
6,0.454,0.711,0.763,0.408,0.446,0.502,0.535,0.305,0.554,0.249
7,0.329,0.534,0.568,0.371,0.585,0.489,0.491,0.847,0.507,0.583
8,0.552,0.290,0.555,0.715,0.538,0.407,0.596,0.545,0.452,0.780
9,0.540,0.568,0.900,0.344,0.259,0.386,0.456,0.567,0.493,0.468


In [12]:
y_data_list[0][0].sample(10)

split423    0
lp549       0
split621    0
feat194     0
feat240     0
split400    0
split982    0
split619    0
split29     0
feat257     0
dtype: int64

In [13]:
y_data_list[1][1].sample(10)

,result
1426,0
4113,1
189,0
1125,1
4360,0
2312,0
4612,0
3547,0
712,0
3138,0


In [14]:
y_data_list[2][1].sample(10)

,result
3323,1
4286,1
2594,0
3908,0
1476,1
4496,1
3171,1
4235,0
4156,0
176,0


In [15]:
y_data_raw_list[2][1].sample(10)

,result_raw
2255,1
3883,1
3986,1
3047,1
1055,0
2494,0
3640,0
1850,1
4276,0
1560,0


In [16]:
try:
    print(distribution_parameter_list_list[2][1])
except:
    pass

In [17]:
y_data_list[1][0].values

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
path_functions = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'
function_df.to_csv(path_functions, index=False)

path_X_data = './data/saved_function_lists/X_data_' + path_identifier_function_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)
    
path_y_data = './data/saved_function_lists/y_data_' + path_identifier_function_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)

try:
    path_distribution = './data/saved_function_lists/distribution_parameter_list_list_' + path_identifier_function_data + '.pkl'
    with open(path_distribution, 'wb') as f:
        pickle.dump(distribution_parameter_list_list, f)
except:
    pass